## Este es para correr los datos de los archivos csv generados con c++
## Sebastián Alí Sacasa Céspedes 09/11/2025

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from pathlib import Path

def plot_quantum_results():
    """
    Función principal para graficar todos los resultados del solver cuántico
    """
    # Configuración de estilo para los gráficos
    plt.style.use('default')
    plt.rcParams['figure.figsize'] = [12, 8]
    plt.rcParams['font.size'] = 12
    
    # Verificar que los archivos existan
    data_files = {
        'tiempos': 'tiempos.csv',
        'magnetizacion': 'magnetizacion.csv'
    }
    
    missing_files = []
    for name, file in data_files.items():
        if not os.path.exists(file):
            missing_files.append(file)
    
    if missing_files:
        print(f" Archivos faltantes en : {missing_files}")
        print("Asegúrate de que el programa C++ haya generado los archivos CSV primero.")
        return
    
    print(" Cargando y graficando datos del solver cuántico...")
    
    try:
        # Cargar datos
        tiempos = pd.read_csv('tiempos.csv', header=None).iloc[:, 0].values
        magnetizacion = pd.read_csv('magnetizacion.csv', header=None).iloc[:, 0].values
        
        # Crear figura con subgráficos
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle('Resultados del Solver Cuántico - Modelo de Ising Transversal', 
                    fontsize=16, fontweight='bold')
        
        # Gráfico 1 Evolución temporal de la magnetización
        axes[0, 0].plot(tiempos, magnetizacion, 'b-', linewidth=2, label='Magnetización')
        axes[0, 0].plot(tiempos, magnetizacion, 'ro', markersize=4, alpha=0.6)
        axes[0, 0].set_xlabel('Tiempo')
        axes[0, 0].set_ylabel('Magnetización $\\langle S_z \\rangle / N$')
        axes[0, 0].set_title('Evolución Temporal de la Magnetización')
        axes[0, 0].grid(True, alpha=0.3)
        axes[0, 0].legend()
        
        # Gráfico 2 Valor absoluto de la magnetización (para ver oscilaciones)
        axes[0, 1].plot(tiempos, np.abs(magnetizacion), 'g-', linewidth=2)
        axes[0, 1].set_xlabel('Tiempo')
        axes[0, 1].set_ylabel('$|\\langle S_z \\rangle| / N$')
        axes[0, 1].set_title('Valor Absoluto de la Magnetización')
        axes[0, 1].grid(True, alpha=0.3)
        
        # Gráfico 3 Derivada numérica (tasa de cambio)
        if len(tiempos) > 1:
            derivada = np.gradient(magnetizacion, tiempos)
            axes[1, 0].plot(tiempos, derivada, 'purple', linewidth=2)
            axes[1, 0].set_xlabel('Tiempo')
            axes[1, 0].set_ylabel('$d\\langle S_z \\rangle/dt$')
            axes[1, 0].set_title('Derivada de la Magnetización')
            axes[1, 0].grid(True, alpha=0.3)
        
        # Gráfico 4 Análisis espectral (FFT)
        if len(tiempos) > 10:
            # Calcular FFT para ver frecuencias dominantes
            dt = tiempos[1] - tiempos[0] if len(tiempos) > 1 else 1.0
            fft_result = np.fft.fft(magnetizacion - np.mean(magnetizacion))
            freqs = np.fft.fftfreq(len(tiempos), dt)
            
            # Solo frecuencias positivas
            positive_freq_idx = freqs > 0
            freqs_positive = freqs[positive_freq_idx]
            fft_magnitude = np.abs(fft_result[positive_freq_idx])
            
            axes[1, 1].plot(freqs_positive, fft_magnitude, 'orange', linewidth=2)
            axes[1, 1].set_xlabel('Frecuencia')
            axes[1, 1].set_ylabel('Amplitud FFT')
            axes[1, 1].set_title('Espectro de Frecuencias (FFT)')
            axes[1, 1].grid(True, alpha=0.3)
            axes[1, 1].set_xlim(0, min(10, np.max(freqs_positive)))  # Limitar rango para mejor visualización
        
        plt.tight_layout()
        plt.savefig('resultados_completos.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # Gráfico adicional Solo la magnetización con más detalles
        plt.figure(figsize=(12, 6))
        plt.plot(tiempos, magnetizacion, 'b-', linewidth=3, label='Magnetización', alpha=0.8)
        plt.plot(tiempos, magnetizacion, 'ro', markersize=5, alpha=0.6)
        
        # Añadir líneas de referencia
        plt.axhline(y=0, color='k', linestyle='--', alpha=0.3, label='Cero')
        plt.axhline(y=np.mean(magnetizacion), color='r', linestyle='--', 
                   alpha=0.5, label=f'Promedio: {np.mean(magnetizacion):.3f}')
        
        # Resaltar valores máximo y mínimo
        max_idx = np.argmax(magnetizacion)
        min_idx = np.argmin(magnetizacion)
        
        plt.plot(tiempos[max_idx], magnetizacion[max_idx], 'g^', markersize=10, 
                label=f'Máx: {magnetizacion[max_idx]:.3f}')
        plt.plot(tiempos[min_idx], magnetizacion[min_idx], 'mv', markersize=10, 
                label=f'Mín: {magnetizacion[min_idx]:.3f}')
        
        plt.xlabel('Tiempo')
        plt.ylabel('Magnetización $\\langle S_z \\rangle / N$')
        plt.title('Evolución Temporal de la Magnetización - Modelo de Ising')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.savefig('magnetizacion_detallada.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # Mostrar estadísticas
        print("\n Estadísticas de la Magnetización")
        print(f"    Valor inicial: {magnetizacion[0]:.4f}")
        print(f"   Valor final: {magnetizacion[-1]:.4f}")
        print(f"    Promedio: {np.mean(magnetizacion):.4f}")
        print(f"    Desviación estándar: {np.std(magnetizacion):.4f}")
        print(f"    Mínimo: {np.min(magnetizacion):.4f}")
        print(f"   Máximo: {np.max(magnetizacion):.4f}")
        print(f"    Rango: {np.ptp(magnetizacion):.4f}")
        
        # Análisis de oscilaciones
        zero_crossings = np.where(np.diff(np.sign(magnetizacion)))[0]
        print(f"    Cruces por cero: {len(zero_crossings)}")
        
        if len(zero_crossings) > 1:
            period_estimate = np.mean(np.diff(tiempos[zero_crossings])) * 2
            print(f"    Período estimado: {period_estimate:.4f}")
        
        print(f"\n Gráficos guardados como:")
        print("   - resultados_completos.png")
        print("   - magnetizacion_detallada.png")
        
    except Exception as e:
        print(f" Error al procesar los datos {e}")
        print("Verifica, por favor, que los archivos CSV tengan el formato correcto.")

def check_for_additional_files():
    """
    Función para buscar y graficar archivos adicionales que pueda generar el código C++
    """
    additional_files = []
    
    # Buscar archivos CSV que no sean los principales
    for file in Path('.').glob('*.csv'):
        if file.name not in ['tiempos.csv', 'magnetizacion.csv']:
            additional_files.append(file.name)
    
    if additional_files:
        print(f"\n Archivos adicionales encontrados: {additional_files}")
        
        for file in additional_files:
            try:
                data = pd.read_csv(file)
                plt.figure(figsize=(10, 6))
                
                if data.shape[1] == 1:
                    # Es un vector
                    plt.plot(data.iloc[:, 0], 'b-', linewidth=2)
                    plt.title(f'Datos de {file}')
                    plt.xlabel('Índice')
                    plt.ylabel('Valor')
                elif data.shape[1] == 2:
                    # Son datos complejos (parte real e imaginaria)
                    plt.plot(data.iloc[:, 0], label='Parte Real', linewidth=2)
                    plt.plot(data.iloc[:, 1], label='Parte Imaginaria', linewidth=2)
                    plt.title(f'Datos Complejos de {file}')
                    plt.xlabel('Índice')
                    plt.ylabel('Valor')
                    plt.legend()
                else:
                    # Es una matriz
                    plt.imshow(data.values, cmap='viridis', aspect='auto')
                    plt.colorbar(label='Valor')
                    plt.title(f'Matriz de {file}')
                    plt.xlabel('Columna')
                    plt.ylabel('Fila')
                
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.savefig(f'{file}_plot.png', dpi=300, bbox_inches='tight')
                plt.show()
                
                print(f"  Gráfico generado para: {file}")
                
            except Exception as e:
                print(f"   Error al procesar {file}: {e}")

if __name__ == "__main__":
    print("Vuelve a graficar los resultados del solver cuántico del Modelo de Ising ")
    print("Cargando y analizando datos del solver...")
    
    # Graficar resultados principales
    plot_quantum_results()
    
    # Buscar y graficar archivos adicionales
    check_for_additional_files()
    
    print("\n Proceso de visualización completado, pura vida")